In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install pybamm


In [3]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import pybamm


ERROR:2026-01-25 17:26:22,510:jax._src.xla_bridge:487: Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/xla_bridge.py", line 485, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/usr/local/lib/python3.12/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 328, in initialize
    _check_cuda_versions(raise_on_first_error=True)
  File "/usr/local/lib/python3.12/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 285, in _check_cuda_versions
    local_device_count = cuda_versions.cuda_device_count()
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: jaxlib/cuda/versions_helpers.cc:113: operation cuInit(0) failed: Unknown CUDA error 303; cuGetErrorName failed. This probably means that JAX was unable to load the CUDA libraries.


In [12]:
# Dataset
DATA_PATH = "/kaggle/input/nasa-battery-dataset"
V_CUTOFF = 2.7
NOMINAL_AH = 2.0

# Resolutions
BINS = 20      # Transformer
M_PHYS = 200   # Physics grid

# Training
BATCH_SIZE = 32
EPOCHS = 20
LR = 1e-3
LAMBDA_PHYS = 0.05

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [14]:
metadata = pd.read_csv(
    os.path.join(DATA_PATH, "cleaned_dataset", "metadata.csv")
)

metadata["battery_id"] = metadata["battery_id"].astype(str)

excluded = ["B0049", "B0050", "B0051", "B0052"]

metadata = metadata[
    (metadata["type"] == "discharge") &
    (~metadata["battery_id"].isin(excluded))
]

metadata["cycle_number"] = metadata.groupby("battery_id").cumcount() + 1


In [16]:
main_rows = []
phys_rows = []

for _, row in tqdm(metadata.iterrows(), total=len(metadata)):
    df = pd.read_csv(
        os.path.join(DATA_PATH, "cleaned_dataset", "data", row["filename"])
    )

    df = df.sort_values("Time").drop_duplicates("Time")
    df = df[df["Voltage_measured"] >= V_CUTOFF]

    if len(df) < 10:
        continue

    # Capacity
    df["dt_hr"] = df["Time"].diff().fillna(0) / 3600
    df["dQ"] = df["Current_measured"] * df["dt_hr"]
    capacity = abs(df["dQ"].sum())

    if capacity <= 0.8 or capacity >= NOMINAL_AH:
        continue

    df["SoC"] = 100 * (1 + df["dQ"].cumsum() / capacity)
    soh = 100 * capacity / NOMINAL_AH

    # Resample
    t = df["Time"].values
    t_grid = np.linspace(t[0], t[-1], M_PHYS)
    dt_hr = np.diff(t_grid, prepend=t_grid[0]) / 3600

    def interp(col):
        return np.interp(t_grid, t, df[col])

    V = interp("Voltage_measured")
    I = interp("Current_measured")
    T = interp("Temperature_measured")
    S = interp("SoC")

    # Physics dataset
    for k in range(M_PHYS):
        phys_rows.append({
            "battery_id": row["battery_id"],
            "cycle_number": row["cycle_number"],
            "t_idx": k,
            "dt_hr": dt_hr[k],
            "V": V[k],
            "I": I[k],
            "T": T[k],
            "SoC": S[k],
            "Capacity": capacity
        })

    # Transformer bins
    rs = pd.DataFrame({"V": V, "I": I, "T": T, "SoC": S})
    chunks = np.array_split(rs, BINS)

    for i, c in enumerate(chunks):
        main_rows.append({
            "battery_id": row["battery_id"],
            "cycle_number": row["cycle_number"],
            "bin": i,
            "V": c["V"].mean(),
            "I": c["I"].mean(),
            "T": c["T"].mean(),
            "SoC": c["SoC"].mean(),
            "SoH": soh
        })


  0%|          | 0/2694 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.1

In [17]:
main_df = pd.DataFrame(main_rows)
phys_df = pd.DataFrame(phys_rows)

main_df.to_csv("main_20bins.csv", index=False)
phys_df.to_csv("phys_200pts.csv", index=False)


In [55]:
def run_spm_voltage(I, T, time_s):
    model = pybamm.lithium_ion.SPM()
    params = pybamm.ParameterValues("Chen2020")

    params.update({
        "Current function [A]": pybamm.Interpolant(
            time_s, I, pybamm.t
        )
    })

    sim = pybamm.Simulation(model, parameter_values=params)
    sim.solve(t_eval=time_s)

    return sim.solution["Terminal voltage [V]"](time_s)


In [56]:
FEATURES = ["V", "I", "T", "SoC"]

X, y = [], []

for (_, _), g in main_df.groupby(["battery_id", "cycle_number"]):
    g = g.sort_values("bin")
    if len(g) != BINS:
        continue
    X.append(g[FEATURES].values)
    y.append(g["SoH"].iloc[0])

X = np.array(X)
y = np.array(y)

scaler = StandardScaler()
X = scaler.fit_transform(X.reshape(-1, 4)).reshape(-1, BINS, 4)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [58]:
class BatteryDataset(Dataset):
    def __init__(self, X, y, ids):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.ids = ids  # list of (bid, cycle)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.ids[idx]


In [60]:
class SoHTransformer(nn.Module):
    def __init__(self):
        super().__init__()

        self.embed = nn.Linear(4, 64)

        encoder = nn.TransformerEncoderLayer(
            d_model=64, nhead=4, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder, 2)

        # Heads
        self.soh_head = nn.Linear(64, 1)
        self.r_head = nn.Linear(64, 1)   # resistance scale

    def forward(self, x):
        x = self.embed(x)
        x = self.encoder(x)
        h = x.mean(dim=1)

        soh = self.soh_head(h).squeeze()
        r_scale = torch.sigmoid(self.r_head(h)) + 0.5

        return soh, r_scale


In [67]:
def spm_voltage_loss(model, phys_df, r_scale):
    losses = []

    for (_, _), g in phys_df.groupby(["battery_id", "cycle_number"]):
        I = g["I"].values
        T = g["T"].values
        t = np.cumsum(g["dt_hr"].values) * 3600

        try:
            V_spm = run_spm_voltage(I, T, t)
        except:
            continue

        V_meas = torch.tensor(g["V"].values, dtype=torch.float32)
        V_spm = torch.tensor(V_spm, dtype=torch.float32)

        losses.append(torch.mean((r_scale.mean() * V_spm - V_meas) ** 2))

    return torch.mean(torch.stack(losses))


In [68]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LR = 1e-3
BATCH_SIZE = 32

model = SoHTransformer().to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)
mse = nn.MSELoss()

train_loader = DataLoader(
    BatteryDataset(X_train, y_train),
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_loader = DataLoader(
    BatteryDataset(X_val, y_val),
    batch_size=BATCH_SIZE,
    shuffle=False
)


TypeError: BatteryDataset.__init__() missing 1 required positional argument: 'ids'

In [69]:

spm_cache = {}

for (bid, cycle), g in phys_df.groupby(["battery_id", "cycle_number"]):
    I = g["I"].values
    t = np.cumsum(g["dt_hr"].values) * 3600

    try:
        V_spm = run_spm_voltage(I, None, t)
        spm_cache[(bid, cycle)] = torch.tensor(V_spm, dtype=torch.float32)
    except:
        continue


In [70]:
torch.save(spm_cache, "spm_cache.pt")


In [75]:
spm_cache = torch.load(
    "spm_cache.pt",
    map_location=DEVICE,
    weights_only=False
)


for epoch in range(10):
    model.train()
    total = 0

    for xb, yb, ids in train_loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)

        opt.zero_grad()

        soh_pred, r = model(xb)
        loss_data = mse(soh_pred, yb)

        loss_phys = 0
        for i, (bid, cycle) in enumerate(ids):
            if (bid, cycle) not in spm_cache:
                continue

            V_spm = spm_cache[(bid, cycle)].to(DEVICE)
            V_meas = phys_tensor[(bid, cycle)].to(DEVICE)

            loss_phys += torch.mean((r[i] * V_spm - V_meas) ** 2)

        loss_phys /= len(ids)

        loss = loss_data + LAMBDA_PHYS * loss_phys
        loss.backward()
        opt.step()

        total += loss.item()

    print(f"Epoch {epoch+1:02d} | Loss {total/len(train_loader):.3f}")


NameError: name 'train_loader' is not defined

In [73]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(DEVICE)
        pred, _ = model(xb)
        y_pred.extend(pred.cpu().numpy())
        y_true.extend(yb.numpy())

plt.figure(figsize=(6,6))
plt.scatter(y_true, y_pred, alpha=0.6)
plt.plot([min(y_true), max(y_true)],
         [min(y_true), max(y_true)], 'r--')
plt.xlabel("True SoH")
plt.ylabel("Predicted SoH")
plt.title("Transformer + SPM PINN")
plt.grid()
plt.show()


NameError: name 'val_loader' is not defined